In [2]:
import os
from datetime import datetime, date
import pandas as pd
import pyodbc
import useful_functions as use
username = 'vigrose'
password_aims = 'DQA8K@T7pXEqEt'
password_edw = 'Ravenclaw~10946'

In [3]:
oneview_dir = 'U:/Source Files/Data Analytics/Data-Science/Data/Masterfile_OneVIew/'
ethnicity_filename = use.get_newest(oneview_dir, 'AIMS_PERSON_ETHNICITY_ET')
ethnicity = pd.read_csv(ethnicity_filename, sep='|')

reference_filename = use.get_newest(oneview_dir, 'AIMS_ETHNICITY_ER')
reference = pd.read_csv(reference_filename, sep='|')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

# username = 'dlabs'
# password_aims = 'yV80tNwNN64q4644'
s = "DSN=aims_prod; UID={}; PWD={}".format(username, password_aims)
informix = pyodbc.connect(s)

In [8]:
ME_ENTITY_QUERY = 'SELECT DISTINCT M.ME, E.ENTITY_ID FROM AMAEDW.PARTY_ID_TO_ME_VW M, AMAEDW.PARTY_ENTITY_VW E WHERE M.PARTY_ID=E.PARTY_ID'
me_entity = pd.read_sql(con=AMAEDW, sql=ME_ENTITY_QUERY)

NameError: name 'me_to_entity' is not defined

In [113]:
tables = [me_entity, ethnicity, reference]

In [112]:
reference.parent_category_desc = [x.strip() for x in reference.parent_category_desc]

def ethnicity_extract():
    me_to_entity = get_me_entity()
    ethnicity = get_person_ethnicity()
    reference = get_reference()
    tables = [me_to_entity, ethnicity, reference]
    return tables

#transform
def find_multiple_identities(ethnicity):
    single_identity = ethnicity[ethnicity.parent_category_desc!='Unknown'].drop_duplicates(['entity_id', 'parent_category_desc'], keep=False)
    multiple_identities = ethnicity[ethnicity.entity_id.isin(~single_identity.entity_id)].drop_duplicates('entity_id')
    multiple_identities['race_ethnicity'] = 'Mixed Race/Ethnicity'
    single_identity['race_ethnicity'] = single_identity.parent_category_desc
    return single_identity[['entity_id','race_ethnicity']], multiple_identities[['entity_id','race_ethnicity']]
    
def add_reference(ethnicity, reference):
    ethnicity['ethnicity'] = ethnicity.ethnicity_id.astype(str)
    ethnicity_reference = pd.merge(ethnicity, reference, on='ethnicity')
    return ethnicity_reference

def ethnicity_transform(tables):
    me_to_entity = tables[0]
    me_to_entity['medical_education_number'] = use.fix_me(me_to_entity.ME)
    plz = add_reference(tables[1], tables[2])
    single_identity, multi_identity = find_multiple_identities(plz)
    race_ethnicity = pd.concat([single_identity, multi_identity])
    race_ethnicity['ENTITY_ID'] = race_ethnicity.entity_id.astype(str)
    race_ethnicity = pd.merge(race_ethnicity, me_to_entity, on='ENTITY_ID')
    race_ethnicity = race_ethnicity[['medical_education_number','race_ethnicity']]
    return race_ethnicity

In [114]:
race_ethnicity = ethnicity_transform(tables)

In [12]:
me_to_entity = tables[0]
me_to_entity['medical_education_number'] = use.fix_me(me_to_entity.ME)


In [52]:
single_identity, multi_identity = find_multiple_identities(tables[1])

In [50]:
single_identity = add_reference(single_identity, tables[2])

In [19]:
race_ethnicity = pd.concat([single_identity, multi_identity])

In [21]:
race_ethnicity['ENTITY_ID'] = race_ethnicity.entity_id.astype(str)

In [22]:
race_ethnicity = pd.merge(race_ethnicity, me_to_entity, on='ENTITY_ID')

In [24]:
race_ethnicity = race_ethnicity[['medical_education_number','race_ethnicity']]

In [73]:
race_ethnicity.groupby('race_ethnicity').count()

,medical_education_number
race_ethnicity,
American Indian or Alaskan Native,3514
Asian,153929
Asian,8632
Black or African American,48099
"Hispanic, Latino, or Spanish Origin",37506
"Hispanic, Latino, or Spanish Origin",20307
Mixed Race/Ethnicity,8978
Native Hawaiian or Other Pacific Islander,2616
Other,29053


In [26]:
today = str(date.today())
race_ethnicity.head(10).to_csv(f'C:/Users/vigrose/DataPhysicianRaceEthnicity_{today}.psv', sep='|', index=False)
race_ethnicity.head(10).to_csv(f'C:/Users/vigrose/DataPhysicianRaceEthnicity_2_{today}.psv', index=False)

In [ ]:
#load
def ethnicity_load(race_ethnicity):
    oneview_dir = os.environ.get('ONEVIEW_FOLDER')
    today = str(date.today())
    race_ethnicity.to_csv(f'{oneview_dir}PhysicianRaceEthnicity_{today}.psv', sep='|', index=False)
    race_ethnicity.head(10).to_csv(f'C:/Users/vigrose/DataPhysicianRaceEthnicity_{today}.psv', sep='|', index=False)
    race_ethnicity.head(10).to_csv(f'C:/Users/vigrose/DataPhysicianRaceEthnicity_2_{today}.psv', index=False)

def race_ethnicity_etl():
    LOGGER.info('Extract...')
    tables = ethnicity_extract()
    LOGGER.info('Transform...')
    race_ethnicity = ethnicity_transform(tables)
    LOGGER.info('Load...')
    ethnicity_load(race_ethnicity)

In [29]:
TEST = pd.read_csv('../../Data/MasterfileCore/PhysicianRaceEthnicity-ETL.psv', sep='|')

In [161]:
TEST_3 = pd.read_csv('../../Data/MasterfileCore/PhysicianRaceEthnicity_2021-12-08.psv', sep='|')

In [162]:
TEST_3

,medical_education_number,race_ethnicity
0,2701940604,Black or African American
1,4101840684,Black or African American
2,2802840627,Black or African American
3,2802840937,Black or African American
4,2802760259,Black or African American
...,...,...
1433643,3401170528,Tribal Affiliation
1433644,3901170788,Tribal Affiliation
1433645,2512171098,Tribal Affiliation
1433646,3701170121,Tribal Affiliation


In [80]:
len(TEST_2)/len(TEST)

0.7567945535483593

In [34]:
TEST.drop_duplicates('medical_education_number')

,medical_education_number,race_ethnicity
0,102010159,White
1,102020146,White
2,102031113,White
3,102031318,White
4,102040414,White
...,...,...
1350746,175210407,Unknown
1350747,3575240291,Unknown
1350748,5175220188,Unknown
1350749,3575240304,Unknown


In [81]:
TEST_2

,medical_education_number,race_ethnicity
0,2701940604,Black or African American
1,4101840684,Black or African American
2,2802840627,Black or African American
3,2802840937,Black or African American
4,2802760259,Black or African American
...,...,...
1022236,3401170528,Tribal Affiliation
1022237,3901170788,Tribal Affiliation
1022238,2512171098,Tribal Affiliation
1022239,3701170121,Tribal Affiliation


In [71]:
me_entity

,ME,ENTITY_ID,medical_education_number
0,04501810245,10000012,04501810245
1,04501810253,10000025,04501810253
2,04501810270,10000038,04501810270
3,04501810288,10000051,04501810288
4,04501810296,10000064,04501810296
...,...,...,...
1767278,04501810199,9999947,04501810199
1767279,04501810202,9999960,04501810202
1767280,04501810211,9999973,04501810211
1767281,04501810229,9999986,04501810229


In [37]:
me_entity

,ME,ENTITY_ID,medical_education_number
0,04501810245,10000012,04501810245
1,04501810253,10000025,04501810253
2,04501810270,10000038,04501810270
3,04501810288,10000051,04501810288
4,04501810296,10000064,04501810296
...,...,...,...
1767278,04501810199,9999947,04501810199
1767279,04501810202,9999960,04501810202
1767280,04501810211,9999973,04501810211
1767281,04501810229,9999986,04501810229


In [51]:
single_identity[ethnicity.entity_id.isin(me_entity.ENTITY_ID)].drop_duplicates('entity_id')

,entity_id,race_ethnicity
0,62912694,Unknown
1,62912759,Unknown
2,62912837,Unknown
3,62912915,Unknown
4,62913032,Unknown
...,...,...
571142,67782546,Native Hawaiian or Other Pacific Islander
571143,68461471,Native Hawaiian or Other Pacific Islander
571144,66901666,Native Hawaiian or Other Pacific Islander
571145,68325816,Native Hawaiian or Other Pacific Islander


In [45]:
len(multi_identity)

8978

In [46]:
tables[1]

,entity_id,ethnicity_id,active_ind,preferred_ind,reported_dt,src_category,src_cat_code,insert_user_id,insert_dtm,update_user_id,update_dtm,opt_lock
0,6013952,2,Y,Y,07/30/2002,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,08/20/2021 12:12:01 PM,4
1,6014069,1,Y,Y,07/30/2002,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,08/20/2021 12:12:01 PM,4
2,6014186,1,Y,Y,07/30/2002,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,08/20/2021 12:12:01 PM,4
3,6014303,1,Y,Y,07/30/2002,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,08/20/2021 12:12:01 PM,4
4,6014420,1,Y,Y,07/30/2002,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,08/20/2021 12:12:01 PM,4
...,...,...,...,...,...,...,...,...,...,...,...,...
2310100,69136795,9,Y,Y,12/03/2021,SRC-ETHNIC,MASTERFILE,etlaims,12/04/2021 03:00:40 PM,etlaims,12/04/2021 03:04:33 PM,1
2310101,69136860,9,Y,Y,12/03/2021,SRC-ETHNIC,MASTERFILE,etlaims,12/04/2021 03:00:40 PM,etlaims,12/04/2021 03:04:33 PM,1
2310102,69136899,9,Y,Y,12/03/2021,SRC-ETHNIC,MASTERFILE,etlaims,12/04/2021 03:00:40 PM,etlaims,12/04/2021 03:04:33 PM,1
2310103,69136925,9,Y,Y,12/03/2021,SRC-ETHNIC,MASTERFILE,etlaims,12/04/2021 03:00:40 PM,etlaims,12/04/2021 03:04:33 PM,1


In [53]:
pd.merge(single_identity, reference, left_on='ethnicity_id', right_on='ethnicity')

,entity_id,ethnicity_id,active_ind_x,preferred_ind,reported_dt,src_category,src_cat_code,insert_user_id_x,insert_dtm_x,update_user_id_x,...,ethnicity,description,active_ind_y,source_detail_code,parent_category_desc,insert_user_id_y,insert_dtm_y,update_user_id_y,update_dtm_y,opt_lock_y
0,62912694,9,Y,Y,10/28/2020,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 06:04:29 PM,etlaims,...,9,Unknown,Y,U00,Unknown,informix,10/22/2000 06:07:58 AM,skolli,07/08/2021 04:28:53 PM,3
1,62912759,9,Y,Y,10/28/2020,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 06:04:29 PM,etlaims,...,9,Unknown,Y,U00,Unknown,informix,10/22/2000 06:07:58 AM,skolli,07/08/2021 04:28:53 PM,3
2,62912837,9,Y,Y,08/20/2021,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 06:04:29 PM,etlaims,...,9,Unknown,Y,U00,Unknown,informix,10/22/2000 06:07:58 AM,skolli,07/08/2021 04:28:53 PM,3
3,62912915,9,Y,Y,10/28/2020,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 06:04:29 PM,etlaims,...,9,Unknown,Y,U00,Unknown,informix,10/22/2000 06:07:58 AM,skolli,07/08/2021 04:28:53 PM,3
4,62913032,9,Y,Y,10/28/2020,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 06:04:29 PM,etlaims,...,9,Unknown,Y,U00,Unknown,informix,10/22/2000 06:07:58 AM,skolli,07/08/2021 04:28:53 PM,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571142,67782546,e,Y,Y,07/20/2021,SRC-ETHNIC,AAMC,etlaims,11/27/2021 03:03:41 PM,etlaims,...,e,Samoan,Y,P06,Native Hawaiian or Other Pacific Islander,skolli,06/14/2021 11:19:58 AM,murahman,07/15/2021 10:35:18 AM,1
571143,68461471,e,Y,Y,07/28/2021,SRC-ETHNIC,AAMC,etlaims,11/27/2021 03:03:42 PM,etlaims,...,e,Samoan,Y,P06,Native Hawaiian or Other Pacific Islander,skolli,06/14/2021 11:19:58 AM,murahman,07/15/2021 10:35:18 AM,1
571144,66901666,e,Y,Y,07/06/2021,SRC-ETHNIC,AAMC,etlaims,11/27/2021 03:03:42 PM,etlaims,...,e,Samoan,Y,P06,Native Hawaiian or Other Pacific Islander,skolli,06/14/2021 11:19:58 AM,murahman,07/15/2021 10:35:18 AM,1
571145,68325816,e,Y,Y,08/02/2021,SRC-ETHNIC,AAMC,etlaims,11/27/2021 03:03:44 PM,etlaims,...,e,Samoan,Y,P06,Native Hawaiian or Other Pacific Islander,skolli,06/14/2021 11:19:58 AM,murahman,07/15/2021 10:35:18 AM,1


In [106]:
ethnicity[(ethnicity.entity_id.isin(me_entity.ENTITY_ID))&(~ethnicity.entity_id.isin(TEST_2.ENTITY_ID))].drop_duplicates('entity_id').ethnicity_id.unique()

array([9, 4, 2, 1, 3, 7, 5, '9', '1', '2', '5', '4', '3', '7'],
      dtype=object)

In [94]:
TEST_2['ME'] = use.fix_me(TEST_2.medical_education_number)

In [96]:
TEST_2 = pd.merge(TEST_2, me_entity, on='ME')

In [98]:
TEST_2

,medical_education_number_x,race_ethnicity,ME,ENTITY_ID,medical_education_number_y
0,2701940604,Black or African American,02701940604,6013952,02701940604
1,4101840684,Black or African American,04101840684,9053027,04101840684
2,2802840627,Black or African American,02802840627,6099947,02802840627
3,2802840937,Black or African American,02802840937,6100298,02802840937
4,2802760259,Black or African American,02802760259,6085049,02802760259
...,...,...,...,...,...
1022239,3401170528,Tribal Affiliation,03401170528,46013119,03401170528
1022240,3901170788,Tribal Affiliation,03901170788,46000587,03901170788
1022241,2512171098,Tribal Affiliation,02512171098,46148306,02512171098
1022242,3701170121,Tribal Affiliation,03701170121,45956309,03701170121


,ME,ENTITY_ID,medical_education_number
0,04501810245,10000012,04501810245
1,04501810253,10000025,04501810253
2,04501810270,10000038,04501810270
3,04501810288,10000051,04501810288
4,04501810296,10000064,04501810296
...,...,...,...
1767278,04501810199,9999947,04501810199
1767279,04501810202,9999960,04501810202
1767280,04501810211,9999973,04501810211
1767281,04501810229,9999986,04501810229


In [103]:
reference.ethnicity.unique()

array(['0', '1', '2', '3', '4', '5', '6', ' ', '7', '8', '9', 'A', 'B',
       'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O',
       'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b',
       'c', 'd', 'e', 'f'], dtype=object)

In [104]:
ethnicity.ethnicity_id.unique()

array([2, 1, 9, 4, 3, 5, 7, '9', '1', '2', '5', '4', '3', '7', 'A', 'D',
       'O', 'H', 'S', 'Y', 'F', 'V', 'T', 'N', 'I', 'K', 'U', 'W', 'M',
       'E', 'P', 'a', 'd', 'B', 'L', 'R', 'c', 'X', 'J', 'Q', 'b', 'C',
       'Z', 'G', 'e'], dtype=object)

In [105]:
reference

,ethnicity,description,active_ind,source_detail_code,parent_category_desc,insert_user_id,insert_dtm,update_user_id,update_dtm,opt_lock
0,0,Unknown,N,U00,Unknown,informix,10/22/2000 06:07:58 AM,skolli,07/08/2021 04:28:53 PM,3
1,1,White ...,Y,W00,White,cmathews,06/19/2020 12:39:56 PM,skolli,06/10/2021 09:56:22 AM,3
2,2,Black or African American ...,Y,B00,Black or African American,cmathews,06/19/2020 12:40:10 PM,skolli,06/10/2021 09:56:22 AM,3
3,3,"Hispanic, Latino, or Spanish Origin ...",Y,H00,"Hispanic, Latino, or Spanish Origin",cmathews,06/19/2020 12:40:24 PM,skolli,06/10/2021 09:59:34 AM,3
4,4,Asian ...,Y,A00,Asian,cmathews,06/19/2020 12:40:38 PM,skolli,06/10/2021 09:59:34 AM,3
5,5,Other,Y,X00,Other,informix,10/22/2000 06:07:58 AM,skolli,06/10/2021 09:59:34 AM,2
6,6,American Indian ...,N,I00,American Indian or Alaskan Native,cmathews,06/19/2020 12:41:04 PM,murahman,07/19/2021 03:39:57 PM,6
7,,Unknown,N,U00,Unknown,informix,10/22/2000 06:07:58 AM,skolli,07/08/2021 04:28:53 PM,3
8,7,American Indian or Alaskan Native ...,Y,I00,American Indian or Alaskan Native,cmathews,06/19/2020 12:40:58 PM,murahman,07/19/2021 03:39:57 PM,5
9,8,Other ...,N,X00,Other,informix,10/22/2000 06:07:58 AM,skolli,07/08/2021 04:30:19 PM,8


In [115]:
race_ethnicity

,medical_education_number,race_ethnicity
0,02701940604,Black or African American
1,04101840684,Black or African American
2,02802840627,Black or African American
3,02802840937,Black or African American
4,02802760259,Black or African American
...,...,...
1022236,03401170528,Tribal Affiliation
1022237,03901170788,Tribal Affiliation
1022238,02512171098,Tribal Affiliation
1022239,03701170121,Tribal Affiliation


In [111]:
TEST_2

,medical_education_number_x,race_ethnicity,ME,ENTITY_ID,medical_education_number_y
0,2701940604,Black or African American,02701940604,6013952,02701940604
1,4101840684,Black or African American,04101840684,9053027,04101840684
2,2802840627,Black or African American,02802840627,6099947,02802840627
3,2802840937,Black or African American,02802840937,6100298,02802840937
4,2802760259,Black or African American,02802760259,6085049,02802760259
...,...,...,...,...,...
1022239,3401170528,Tribal Affiliation,03401170528,46013119,03401170528
1022240,3901170788,Tribal Affiliation,03901170788,46000587,03901170788
1022241,2512171098,Tribal Affiliation,02512171098,46148306,02512171098
1022242,3701170121,Tribal Affiliation,03701170121,45956309,03701170121


In [116]:
plz = add_reference(ethnicity, reference)
single_identity, multi_identity = find_multiple_identities(plz)
# race_ethnicity = pd.concat([single_identity, multi_identity])
# race_ethnicity['ENTITY_ID'] = race_ethnicity.entity_id.astype(str)
# race_ethnicity = pd.merge(race_ethnicity, me_to_entity, on='ENTITY_ID')
# race_ethnicity = race_ethnicity[['medical_education_number','race_ethnicity']]

In [118]:
len(multi_identity)

0

In [121]:
ethnicity[~ethnicity.entity_id.isin(single_identity.entity_id)].drop_duplicates('entity_id')

,entity_id,ethnicity_id,active_ind,preferred_ind,reported_dt,src_category,src_cat_code,insert_user_id,insert_dtm,update_user_id,update_dtm,opt_lock,ethnicity
13,6015941,9,Y,Y,11/29/1999,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,08/20/2021 12:12:01 PM,4,9
22,6016994,9,Y,Y,11/29/1999,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,08/20/2021 12:12:01 PM,4,9
23,6017111,9,Y,Y,11/29/1999,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,08/20/2021 12:12:01 PM,4,9
37,6018983,9,Y,Y,11/29/1999,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,08/20/2021 12:12:01 PM,4,9
47,6020621,9,Y,Y,11/29/1999,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,08/20/2021 12:12:01 PM,4,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2310100,69136795,9,Y,Y,12/03/2021,SRC-ETHNIC,MASTERFILE,etlaims,12/04/2021 03:00:40 PM,etlaims,12/04/2021 03:04:33 PM,1,9
2310101,69136860,9,Y,Y,12/03/2021,SRC-ETHNIC,MASTERFILE,etlaims,12/04/2021 03:00:40 PM,etlaims,12/04/2021 03:04:33 PM,1,9
2310102,69136899,9,Y,Y,12/03/2021,SRC-ETHNIC,MASTERFILE,etlaims,12/04/2021 03:00:40 PM,etlaims,12/04/2021 03:04:33 PM,1,9
2310103,69136925,9,Y,Y,12/03/2021,SRC-ETHNIC,MASTERFILE,etlaims,12/04/2021 03:00:40 PM,etlaims,12/04/2021 03:04:33 PM,1,9


In [123]:
ethnicity.drop_duplicates('entity_id')
#1763871

,entity_id,ethnicity_id,active_ind,preferred_ind,reported_dt,src_category,src_cat_code,insert_user_id,insert_dtm,update_user_id,update_dtm,opt_lock,ethnicity
0,6013952,2,Y,Y,07/30/2002,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,08/20/2021 12:12:01 PM,4,2
1,6014069,1,Y,Y,07/30/2002,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,08/20/2021 12:12:01 PM,4,1
2,6014186,1,Y,Y,07/30/2002,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,08/20/2021 12:12:01 PM,4,1
3,6014303,1,Y,Y,07/30/2002,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,08/20/2021 12:12:01 PM,4,1
4,6014420,1,Y,Y,07/30/2002,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,08/20/2021 12:12:01 PM,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2310100,69136795,9,Y,Y,12/03/2021,SRC-ETHNIC,MASTERFILE,etlaims,12/04/2021 03:00:40 PM,etlaims,12/04/2021 03:04:33 PM,1,9
2310101,69136860,9,Y,Y,12/03/2021,SRC-ETHNIC,MASTERFILE,etlaims,12/04/2021 03:00:40 PM,etlaims,12/04/2021 03:04:33 PM,1,9
2310102,69136899,9,Y,Y,12/03/2021,SRC-ETHNIC,MASTERFILE,etlaims,12/04/2021 03:00:40 PM,etlaims,12/04/2021 03:04:33 PM,1,9
2310103,69136925,9,Y,Y,12/03/2021,SRC-ETHNIC,MASTERFILE,etlaims,12/04/2021 03:00:40 PM,etlaims,12/04/2021 03:04:33 PM,1,9


In [143]:
single_identity = ethnicity_reference.drop_duplicates(['entity_id', 'parent_category_desc'], keep=False)
additional = ethnicity_reference[(~ethnicity_reference.entity_id.isin(single_identity.entity_id))&(ethnicity_reference.parent_category_desc!='Unknown')].drop_duplicates('entity_id', keep=False)
single_identity = pd.concat([single_identity, additional])
multiple_identities = ethnicity_reference[~(ethnicity_reference.entity_id.isin(single_identity.entity_id))].drop_duplicates('entity_id')
multiple_identities['race_ethnicity'] = 'Mixed Race/Ethnicity'
single_identity['race_ethnicity'] = single_identity.parent_category_desc

In [125]:
ethnicity['ethnicity'] = ethnicity.ethnicity_id.astype(str)
ethnicity_reference = pd.merge(ethnicity, reference, on='ethnicity')

In [150]:
multiple_identities = ethnicity_reference[~(ethnicity_reference.entity_id.isin(single_identity.entity_id))].drop_duplicates('entity_id')

0

In [134]:
~ethnicity_reference.entity_id.isin(single_identity.entity_id)

0          False
1          False
2           True
3          False
4          False
           ...  
2310100     True
2310101     True
2310102    False
2310103     True
2310104     True
Name: entity_id, Length: 2310105, dtype: bool

In [135]:
single_identity = ethnicity_reference.drop_duplicates(['entity_id', 'parent_category_desc'], keep=False)
single_identity_known = ethnicity_reference[ethnicity_reference.parent_category_desc!='Unknown'].drop_duplicates(['entity_id', 'parent_category_desc'], keep=False)

290247

In [152]:
len(single_identity.drop_duplicates('entity_id'))+len(multiple_identities)

1763871

In [140]:
#if one row is unknowng, ignore and add to single identity table



,entity_id,ethnicity_id,active_ind_x,preferred_ind,reported_dt,src_category,src_cat_code,insert_user_id_x,insert_dtm_x,update_user_id_x,...,ethnicity,description,active_ind_y,source_detail_code,parent_category_desc,insert_user_id_y,insert_dtm_y,update_user_id_y,update_dtm_y,opt_lock_y


In [148]:
ethnicity_reference.drop_duplicates('entity_id')

,entity_id,ethnicity_id,active_ind_x,preferred_ind,reported_dt,src_category,src_cat_code,insert_user_id_x,insert_dtm_x,update_user_id_x,...,ethnicity,description,active_ind_y,source_detail_code,parent_category_desc,insert_user_id_y,insert_dtm_y,update_user_id_y,update_dtm_y,opt_lock_y
0,6013952,2,Y,Y,07/30/2002,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,...,2,Black or African American ...,Y,B00,Black or African American,cmathews,06/19/2020 12:40:10 PM,skolli,06/10/2021 09:56:22 AM,3
1,9053027,2,Y,Y,07/30/2002,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,...,2,Black or African American ...,Y,B00,Black or African American,cmathews,06/19/2020 12:40:10 PM,skolli,06/10/2021 09:56:22 AM,3
2,9090311,2,Y,N,08/21/2014,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,...,2,Black or African American ...,Y,B00,Black or African American,cmathews,06/19/2020 12:40:10 PM,skolli,06/10/2021 09:56:22 AM,3
3,6099947,2,Y,Y,07/31/2009,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,...,2,Black or African American ...,Y,B00,Black or African American,cmathews,06/19/2020 12:40:10 PM,skolli,06/10/2021 09:56:22 AM,3
4,6100298,2,Y,Y,11/29/1999,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,...,2,Black or African American ...,Y,B00,Black or African American,cmathews,06/19/2020 12:40:10 PM,skolli,06/10/2021 09:56:22 AM,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2302649,45366668,a,Y,Y,10/04/2019,SRC-ETHNIC,GME-SURVEY,etlaims,08/20/2021 11:57:53 AM,etlaims,...,a,Native Hawaiian or Other Pacific Islander,Y,P00,Native Hawaiian or Other Pacific Islander,skolli,06/14/2021 11:19:58 AM,murahman,07/15/2021 10:34:30 AM,1
2302919,45783981,a,Y,Y,09/23/2019,SRC-ETHNIC,GME-SURVEY,etlaims,08/20/2021 12:02:44 PM,etlaims,...,a,Native Hawaiian or Other Pacific Islander,Y,P00,Native Hawaiian or Other Pacific Islander,skolli,06/14/2021 11:19:58 AM,murahman,07/15/2021 10:34:30 AM,1
2303012,47600315,a,Y,Y,08/15/2019,SRC-ETHNIC,GME-SURVEY,etlaims,08/20/2021 12:03:01 PM,etlaims,...,a,Native Hawaiian or Other Pacific Islander,Y,P00,Native Hawaiian or Other Pacific Islander,skolli,06/14/2021 11:19:58 AM,murahman,07/15/2021 10:34:30 AM,1
2303057,46864580,a,Y,Y,08/15/2019,SRC-ETHNIC,GME-SURVEY,etlaims,08/20/2021 12:03:10 PM,etlaims,...,a,Native Hawaiian or Other Pacific Islander,Y,P00,Native Hawaiian or Other Pacific Islander,skolli,06/14/2021 11:19:58 AM,murahman,07/15/2021 10:34:30 AM,1


In [151]:
single_identity.drop_duplicates('entity_id')

,entity_id,ethnicity_id,active_ind_x,preferred_ind,reported_dt,src_category,src_cat_code,insert_user_id_x,insert_dtm_x,update_user_id_x,...,description,active_ind_y,source_detail_code,parent_category_desc,insert_user_id_y,insert_dtm_y,update_user_id_y,update_dtm_y,opt_lock_y,race_ethnicity
0,6013952,2,Y,Y,07/30/2002,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,...,Black or African American ...,Y,B00,Black or African American,cmathews,06/19/2020 12:40:10 PM,skolli,06/10/2021 09:56:22 AM,3,Black or African American
1,9053027,2,Y,Y,07/30/2002,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,...,Black or African American ...,Y,B00,Black or African American,cmathews,06/19/2020 12:40:10 PM,skolli,06/10/2021 09:56:22 AM,3,Black or African American
3,6099947,2,Y,Y,07/31/2009,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,...,Black or African American ...,Y,B00,Black or African American,cmathews,06/19/2020 12:40:10 PM,skolli,06/10/2021 09:56:22 AM,3,Black or African American
4,6100298,2,Y,Y,11/29/1999,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,...,Black or African American ...,Y,B00,Black or African American,cmathews,06/19/2020 12:40:10 PM,skolli,06/10/2021 09:56:22 AM,3,Black or African American
5,6085049,2,Y,Y,11/29/1999,SRC-ETHNIC,MASTERFILE,etlaims,08/19/2021 05:47:33 PM,etlaims,...,Black or African American ...,Y,B00,Black or African American,cmathews,06/19/2020 12:40:10 PM,skolli,06/10/2021 09:56:22 AM,3,Black or African American
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309773,46013119,Z,Y,Y,07/01/2013,SRC-ETHNIC,AAMC,etlaims,08/20/2021 12:03:05 PM,etlaims,...,Tribal Affiliation,Y,I01,Tribal Affiliation,skolli,06/14/2021 11:19:58 AM,murahman,07/15/2021 10:34:07 AM,1,Tribal Affiliation
2309776,46000587,Z,Y,Y,08/19/2013,SRC-ETHNIC,AAMC,etlaims,08/20/2021 12:03:18 PM,etlaims,...,Tribal Affiliation,Y,I01,Tribal Affiliation,skolli,06/14/2021 11:19:58 AM,murahman,07/15/2021 10:34:07 AM,1,Tribal Affiliation
2309780,46148306,Z,Y,Y,08/26/2013,SRC-ETHNIC,AAMC,etlaims,08/20/2021 12:03:26 PM,etlaims,...,Tribal Affiliation,Y,I01,Tribal Affiliation,skolli,06/14/2021 11:19:58 AM,murahman,07/15/2021 10:34:07 AM,1,Tribal Affiliation
2309781,45956309,Z,Y,Y,08/05/2013,SRC-ETHNIC,AAMC,etlaims,08/20/2021 12:03:30 PM,etlaims,...,Tribal Affiliation,Y,I01,Tribal Affiliation,skolli,06/14/2021 11:19:58 AM,murahman,07/15/2021 10:34:07 AM,1,Tribal Affiliation


In [153]:
race_ethnicity.to_csv()

,medical_education_number,race_ethnicity
0,02701940604,Black or African American
1,04101840684,Black or African American
2,02802840627,Black or African American
3,02802840937,Black or African American
4,02802760259,Black or African American
...,...,...
1022236,03401170528,Tribal Affiliation
1022237,03901170788,Tribal Affiliation
1022238,02512171098,Tribal Affiliation
1022239,03701170121,Tribal Affiliation


In [155]:
race_ethnicity = pd.concat([single_identity.drop_duplicates('entity_id'), multiple_identities])

race_ethnicity['ENTITY_ID'] = race_ethnicity.entity_id.astype(str)
race_ethnicity = pd.merge(race_ethnicity, me_to_entity, on='ENTITY_ID')
race_ethnicity = race_ethnicity[['medical_education_number','race_ethnicity']]

In [157]:
race_ethnicity.to_csv(f'C:/Users/vigrose/DataPhysicianRaceEthnicity_plz.psv', sep='|', index=False)

In [160]:
race_ethnicity.drop_duplicates('medical_education_number')

,medical_education_number,race_ethnicity
0,02701940604,Black or African American
1,04101840684,Black or African American
2,02802840627,Black or African American
3,02802840937,Black or African American
4,02802760259,Black or African American
...,...,...
1723890,02604242421,Mixed Race/Ethnicity
1723891,00302240957,Mixed Race/Ethnicity
1723892,03201240921,Mixed Race/Ethnicity
1723893,04816241656,Mixed Race/Ethnicity


In [163]:
len(me_entity)

1767283

In [184]:
this_ethnicity = ethnicity[['entity_id','parent_category_desc']]

In [185]:
this_ethnicity = this_ethnicity.drop_duplicates()

In [187]:
SINGLE = this_ethnicity.drop_duplicates('entity_id', keep=False)

In [172]:
len(single_identity)

1545819

In [178]:
ethnicity = ethnicity[(ethnicity.active_ind=='Y')&(ethnicity.preferred_ind=='Y')]

In [183]:
ethnicity = pd.merge(ethnicity, reference, on='ethnicity', suffixes = (['','_REF']))

In [189]:
this_ethnicity[(~this_ethnicity.entity_id.isin(SINGLE.entity_id))&(this_ethnicity.parent_category_desc!='Unknown')].drop_duplicates('entity_id', keep=False)

,entity_id,parent_category_desc


In [190]:
this_ethnicity[~this_ethnicity.entity_id.isin(SINGLE.entity_id)].drop_duplicates('entity_id')

,entity_id,parent_category_desc
39755,48366080,Black or African American
39759,48432640,Black or African American
39776,63058944,Black or African American
39779,43663616,Black or African American
39781,39733248,Black or African American
...,...,...
1874253,46103287,Asian
1874671,47670788,Black or African American
1875673,46211447,Black or African American
1880336,46207781,Asian


In [191]:
len(SINGLE)+49506

1763971

In [192]:
len(this_ethnicity)

1817040

In [201]:
this_ethnicity[this_ethnicity.entity_id.isin(me_entity.ENTITY_ID)].drop_duplicates('entity_id')

,entity_id,parent_category_desc
0,6013952,Black or African American
1,9053027,Black or African American
2,6099947,Black or African American
3,6100298,Black or African American
4,6085049,Black or African American
...,...,...
1885435,46056162,Asian
1885454,47539189,Asian
1885502,45868819,Native Hawaiian or Other Pacific Islander
1885505,45839907,Native Hawaiian or Other Pacific Islander


In [197]:
me_entity

,ME,ENTITY_ID,medical_education_number
0,04501810245,10000012,04501810245
1,04501810253,10000025,04501810253
2,04501810270,10000038,04501810270
3,04501810288,10000051,04501810288
4,04501810296,10000064,04501810296
...,...,...,...
1767278,04501810199,9999947,04501810199
1767279,04501810202,9999960,04501810202
1767280,04501810211,9999973,04501810211
1767281,04501810229,9999986,04501810229
